In [20]:
# Import ThingsapikeySecret)
import string # for printable comparison
import time # time thread-timeout
import config # for apikeys
import tweepy # for crawling Twitter

In [2]:
# Authenticate Tweepy, a Python Library for crawling Twitter via the Twitter API
# Get Apikey from here: https://developer.twitter.com/en
auth = tweepy.AppAuthHandler(
    config.APIKEY,
    config.SECRET_APIKEY)
api = tweepy.API(auth)

In [3]:
# Get my ID
myName = 'tobiashoelzer'
myID = api.get_user(myName).id

myID

3490529422

In [23]:
# Get my follows (friends)
myFriendsIDs = api.friends_ids(myID)
myFriends = api.friends(myID)

myFriends[0]._json

{'id': 2565039012,
 'id_str': '2565039012',
 'name': 'Diana Ivanova',
 'screen_name': 'diana_nbd',
 'location': 'Leeds, UK',
 'description': 'Research fellow at the School of Earth and Environment, @SEELeeds @CREDS_UK Interested in #Sustainability, #ClimateChange, #SDG, #MRIO, #Consumption',
 'url': 'https://t.co/DVnfVbcezo',
 'entities': {'url': {'urls': [{'url': 'https://t.co/DVnfVbcezo',
     'expanded_url': 'https://environment.leeds.ac.uk/see/staff/2625/dr-diana-ivanova',
     'display_url': 'environment.leeds.ac.uk/see/staff/2625…',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 1188,
 'friends_count': 193,
 'listed_count': 27,
 'created_at': 'Fri Jun 13 10:21:47 +0000 2014',
 'favourites_count': 178,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'verified': False,
 'statuses_count': 251,
 'lang': None,
 'status': {'created_at': 'Tue Oct 20 18:01:09 +0000 2020',
  'id': 1318613239538831362,
  'id_str': '1318

In [24]:
# Get 2. Grade Follows (Friends)
secondGradeFriends = {}
print(f'Start crawling {len(myFriendsIDs)} Friends...')
for f_id in myFriendsIDs: # F is for Friends who do stuff together. U is for You and me. N is for Anywhere and anytime at all. Down here in the deep blue sea!
    try:
        # Get friends of current friend (of mine) and stores them in secondGradeFriends Dictionary
        f_friends = api.friends_ids(f_id)
        secondGradeFriends[f_id] = f_friends
        print(f'Crawled {len(f_friends)} friends for friend with ID: {f_id}')
    except tweepy.RateLimitError:
        # Prevents crashing if the 300 API-Call Limit from Twitters API caused an exeption
        # Tries to continue in 15 Minutes again.
        print(f'Crawled already {len(secondGradeFriends)}!')
        print('Sleep for 15 Minutes to avoid RateLimitErrors')
        time.sleep(15 * 60 + 1)
        f_friends = api.friends_ids(f_id)
        secondGradeFriends[f_id] = f_friends

Start crawling 40 Friends...
Crawled 193 friends for friend with ID: 2565039012
Crawled 1730 friends for friend with ID: 1277617463400828928
Crawled 211 friends for friend with ID: 283258698
Crawled 927 friends for friend with ID: 1094849342
Crawled 466 friends for friend with ID: 894965941066510336
Crawled 817 friends for friend with ID: 3165843875
Crawled 62 friends for friend with ID: 701162514399232006
Crawled 45 friends for friend with ID: 23748536
Crawled 8 friends for friend with ID: 2609199708
Crawled 907 friends for friend with ID: 1103391316464058369
Crawled 1237 friends for friend with ID: 1068337838
Crawled 580 friends for friend with ID: 1112222741548224512
Crawled 368 friends for friend with ID: 3683359935
Crawled 921 friends for friend with ID: 3232960847
Crawled already 14!
Sleep for 15 Minutes to avoid RateLimitErrors
Crawled 865 friends for friend with ID: 1072187272815149057
Crawled 5000 friends for friend with ID: 813286
Crawled 142 friends for friend with ID: 22576

In [25]:
# Reform data to fit into an edge list

# Opens edge file with write access
f = open("my-edgy-friends.edges", "w")
f.write(','.join(['User ID', 'User ID', 'Number of overlapping friends (weight)']) + '\n')

# Array which stores already calculated combinations
matched = []

# I know, there is for sure a better way to do this, but its late and I want to go home. :)
# Double iteration of myFriends
for f_id_i in myFriendsIDs:
    f_friends_i = secondGradeFriends[f_id_i]

    for f_id_j in myFriendsIDs:
        # If i and j are same users or i and j in combination was already calculated continue with next one
        if f_id_j == f_id_i or f'{f_id_j}-{f_id_i}' in matched:
            continue

        f_friends_j = secondGradeFriends[f_id_j]

        # Number which counts the amount of overlapping friends
        same_friends = 0

        # Iterate through friends of i and j to count overlapping friends
        for f_f_id_i in f_friends_i:
            for f_f_id_j in f_friends_j:
                if f_f_id_i == f_f_id_j:
                    same_friends += 1
        
        # If overlapping friends exists write them to the edge list [id of i, id of h, number of overlapping friends]
        if same_friends > 0:
            f.write(','.join([str(f_id_i), str(f_id_j), str(same_friends)]) + '\n')
        
        # Store the combination, so it doesn't calculated twice
        matched.append(f'{f_id_i}-{f_id_j}')

# Close file for os-security
f.close()

In [26]:
# Reform data to fit into an edge list

# Opens edge file with write access
f = open("my-edgy-friend-network.edges", "w")
f.write(','.join(['User ID', 'User ID']) + '\n')

for f_id in secondGradeFriends:
    for f_f_id in secondGradeFriends[f_id]:
        f.write(','.join([str(f_id), str(f_f_id)]) + '\n')

# Close file for os-security
f.close()

In [27]:
# Get Names from friends
f = open("my-nody-friends.nodes", "w")
f.write(','.join(['User ID', 'name', 'screen_name']) + '\n')
print(f'Create Nodelist with {len(myFriends)} Friends...')
for friend in myFriends:
    f_name = ''.join(s for s in friend.name if s in string.printable)
    f.write(','.join([str(friend.id), str(f_name), str(friend.screen_name)]) + '\n')

# Close file for os-security
f.close()

Create Nodelist with 20 Friends...
